In [1]:
import numpy as np
import pandas as pd
import random as rnd
import copy as c

#Pandas options to display max rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

class GameLogic:
    
    def __init__(self):      
        self.constants = self.Constants() #Instantiate constants
        
        tmpPlayer = rnd.choice(self.constants.players) #Instantiate player
        self.player = c.copy(tmpPlayer)
        self.minions = rnd.choices(self.constants.minions, k=rnd.randint(2,4)) #Instantiate minions
        self.boss = rnd.choice(self.constants.bosses) #Instantiate bosses
        
        tmpEnemyList = self.minions + [self.boss] #List of all enemies
        self.enemyList = [c.copy(i) for i in tmpEnemyList]

        self.encounter = pd.DataFrame(None)
        self.killTarget = len(self.enemyList) #The number of enemies player must kill to win
        self.playerTurn = True
        
        #Auto-initiate game turn
        self.turn()

    class Constants:

        def __init__(self):
            #Instantiate players, weapons, and action cards
            self.weapons = (
                self.Weapon('Badass Bullpup', 5),
                self.Weapon('Compound Bow', 3),
                self.Weapon('Alien Wrist Blades', 2),
                self.Weapon('B.F.G.', 10)
            )

            self.cards = (
                self.ActionCard('First Aid Kit', 5, 0, 0, None, 'Get patched up.', True, False),
                self.ActionCard('Coffee', 0, 5, 0, None, 'Nothing like a good cup-o-Joe to keep you going.', True, False),
                self.ActionCard('Quick Workout', 0, 0, 5, None, 'Get pumped up!', True, False),
                self.ActionCard('Super Syrum', 3, 3, 3, None, 'Overall stat boost!', True, False),

                self.ActionCard('Exploding Vehicle', -10, 0, 0, None, 'The vehicle you are driving suddenly erupts in flames. You are very badly burned but still alive… mostly.', True, False),
                self.ActionCard('Let Off Some Steam', -10, 0, 0, None, 'What\'s better than boiling hot water? Boiling hot water vapor.', False, True),
                self.ActionCard('Nice Night for a Walk', -15, 0, 0, None, 'That laundry isn\'t going to do itself. Play this card to take one minion out of play for the remainder of the game.', False, True),
                self.ActionCard('Minor Explosion', -5, 0, 0, None, 'Who put that explosive barrel there?', True, True),
            
                self.ActionCard(self.weapons[0].WPN_name, 0, 0, 0, self.weapons[0], 'WEAPON CARD', True, False),
                self.ActionCard(self.weapons[1].WPN_name, 0, 0, 0, self.weapons[1], 'WEAPON CARD', True, False),
                self.ActionCard(self.weapons[2].WPN_name, 0, 0, 0, self.weapons[2], 'WEAPON CARD', True, False),
                self.ActionCard(self.weapons[3].WPN_name, 0, 0, 0, self.weapons[3], 'WEAPON CARD', True, False)
                
                
            )

            self.players = (
                self.Player('Hollander', 24, 13, 16, None, 'good', 'Veteran Special Operator. Expert in jungle warfare. Not afraid of getting muddy.'),
                self.Player('Doug', 22, 12, 17, None, 'good','Mild-mannered construction worker. Likes demure women. Not a fan of parties.'),
                self.Player('Taskmaster',  23, 12, 17, None, 'good','Veteran Special Agent with an iron-clad secret identity. Sometimes has marital troubles. Not a fan of car salesmen.'),
                self.Player('Vector',  25, 15, 16, None, 'good','Commando by trade. Lumberjack by necessity. Father of the Year.')
            )

            self.minions = (
                self.Player('Sal', 12, 6, 10, None, 'bad', "A funny guy. You should kill him last."), 
                self.Player('Earthquake', 15, 5, 9, None, 'bad',"Nice guy. Throws great parties. Proud of his hands."), 
                self.Player('Mauser', 12, 6, 10, None, 'bad',"Martian Intelligence operative. Not a nice person. Plans great parties though."), 
                self.Player('Svetlana', 13, 5, 9, None, 'bad',"Talented torturer. Known for making her patients talk… unless they pick the lock first."), 
                self.Player('Alien Hunter', 15, 6, 10, None, 'bad',"A hunter from another world who seeks the ultimate prey: you."), 
                self.Player('Killer Robot', 16, 15, 10, None, 'bad',"A cybernetic organism. A killing machine."), 
                self.Player('Paramilitary', 13, 5, 9, None, 'bad',"Drug runners. Kidnappers. Some might call them bad hombres."), 
                self.Player('Corporate Muscle', 14, 8, 8, None, 'bad',"Corporate mercenary. Guaranteed to complicate your day."),
                self.Player('Cerulean Crusader', 11, 5, 7, None, 'bad',"Tired of all the other warm and fuzzy terrorist groups, these extremists formed their own, more extreme group."), 
            )

            self.bosses = (
                self.Player('Copenhaven', 14, 6, 10, None, 'bad',"A corporate dictator with a nasty temper."),
                self.Player('Stevie B', 15, 8, 10, None, 'bad',"A psychotic former commando."),
                self.Player('Alien Boss', 18, 14, 12, None, 'bad',"An alien hunter. The ultimate predator. Loves hot weather."),
                self.Player('Vladimir Von Vlak', 15, 14, 10, None, 'bad',"Radical terrorist by day, equestrian by night."),
                self.Player('DYS-1000', 20, 17, 15, None, 'bad',"A cybernetic organism that can imitate anything it touches. The ultimate mimic. Nigh-unstoppable.")
            )

        class Player:
            def __init__(self, name, HP, SPD, STR, WPN, team, DESC):
                self.name = name
                self.HP = HP #int
                self.SPD = SPD #int
                self.STR = STR #int
                self.WPN = WPN #dictionary with str and float values
                self.team = team #str
                self.DESC = DESC
            cards = []

        class ActionCard:
            def __init__(self, name, HP, SPD, STR, WPN, DESC, affectPlayer, affectEnemy):
                self.name = name
                self.HP = HP
                self.SPD = SPD
                self.STR = STR
                self.WPN = WPN
                self.DESC = DESC
                self.affectPlayer = affectPlayer
                self.affectEnemy = affectEnemy

        class Weapon:        
            def __init__(self, WPN_name, WPN_factor):
                self.WPN_name = WPN_name
                self.WPN_factor = WPN_factor
            description = None
        
    def recordData(self, AP, AE, PC, SR):
        if AP.WPN == None:
            weapon_data = None
        else:
            weapon_data = AP.WPN.WPN_name
            #print(weapon_data) #DEBUG
        
        skirmish = pd.DataFrame([AP.name, AE.name, AP.HP, AP.SPD, 
                                 AP.STR, weapon_data, [card.name for card in AP.cards], 
                                 PC, SR])
        skirmish = skirmish.transpose()
        skirmish.columns = ['Active_Player', 'Active_Enemy', 'AP_HP', 'AP_SPD', 'AP_STR', 'AP_WPN', 
                        'Hand_Cards', 'Played_Cards', 'Match_Result']
        
        self.encounter = pd.concat([self.encounter, skirmish],axis=0)
    
    def turn(self):
        #print(f'\nStarting Kill Target: {self.killTarget}') #DEBUG
        #print(f'Starting Lineup: {[i.name for i in self.enemyList]}')
        while self.killTarget > 0:
            #Check to see whose turn it should be
            if self.playerTurn == True:
                activePlayer = self.player #Map player to local variable
                activeEnemy = self.enemyList[0]
            else:
                activePlayer = self.enemyList[0]
                activeEnemy = self.player

            #Check health of both players
            #print(f'Starting Health:\tActive Player:{activePlayer.HP}\tActive Enemy:{activeEnemy.HP}')
            if activePlayer.HP > 0 and activeEnemy.HP > 0:
                #Draw card
                cardDraw = rnd.randint(0,1)
                if cardDraw == 0:
                    pass
                else:
                    activePlayer.cards.extend(rnd.choices(self.constants.cards, k=rnd.randint(0,2)))
                    
                #if len(activePlayer.cards) == 0:
                    #print(f'{activePlayer.name} did not draw any cards.') #DEBUG
                    #pass #NEWADD
                #else:
                    #print(f'{activePlayer.name} just drew {[c.name for c in activePlayer.cards]}',) #DEBUG
                    #pass #NEWADD
            
                #Play card
                playedCards = []
                if len(activePlayer.cards) == 0:
                    playedCards.append(None)
                    #print('No cards were played this turn.') #DEBUG
                else:
                    #Does card affect both player and enemy?
                    if activePlayer.cards[-1].affectPlayer == True and activePlayer.cards[-1].affectEnemy == True:
                        #print(f'{activePlayer.name} just played {activePlayer.cards[-1].name}')
                        #print('Card Affects: Both Players') #DEBUG
                        activePlayer.HP += activePlayer.cards[-1].HP 
                        activePlayer.SPD += activePlayer.cards[-1].SPD 
                        activePlayer.STR += activePlayer.cards[-1].STR 
                        activeEnemy.HP += activePlayer.cards[-1].HP 
                        activeEnemy.SPD += activePlayer.cards[-1].SPD 
                        activeEnemy.STR += activePlayer.cards[-1].STR 
                        playedCards.append(activePlayer.cards[-1].name) #Add played card to played cards list
                        activePlayer.cards.pop(-1) #Remove card from player's hand
                    elif activePlayer.cards[-1].affectPlayer == True and activePlayer.cards[-1].affectEnemy == False:
                        #print(f'{activePlayer.name} just played {activePlayer.cards[-1].name}')
                        #print('Card Affects: Player') #DEBUG
                        activePlayer.HP += activePlayer.cards[-1].HP 
                        activePlayer.SPD += activePlayer.cards[-1].SPD 
                        activePlayer.STR += activePlayer.cards[-1].STR 
                        if activePlayer.name == self.player.name:
                            activePlayer.WPN = activePlayer.cards[-1].WPN #Assigns/reassigns weapon to player
                        else:
                            activePlayer.WPN = None 
                        playedCards.append(activePlayer.cards[-1].name) #Add played card to played cards list
                        activePlayer.cards.pop(-1) #Remove card from player's hand
                    elif activePlayer.cards[-1].affectPlayer == False and activePlayer.cards[-1].affectEnemy == True:
                        #print(f'{activePlayer.name} just played {activePlayer.cards[-1].name}')
                        #print('Card Affects: Enemy') #DEBUG
                        activeEnemy.HP += activePlayer.cards[-1].HP 
                        activeEnemy.SPD += activePlayer.cards[-1].SPD 
                        activeEnemy.STR += activePlayer.cards[-1].STR 
                        playedCards.append(activePlayer.cards[-1].name) #Add played card to played cards list
                        activePlayer.cards.pop(-1) #Remove card from player's hand
                    elif activePlayer.cards[-1].affectPlayer == False and activePlayer.cards[-1].affectEnemy == False:
                        raise Exception('Card must have effect on either Player or Enemy.')
                    else:
                        raise Exception('Card does not conform to any known paradigm.')

                #Health check
                if activePlayer.HP > 0 and activeEnemy.HP > 0:
                    #print(f'Active Player: {activePlayer.name}\tHP: {activePlayer.HP}') #DEBUG
                    #print(f'Active Enemy: {activeEnemy.name}\tHP: {activeEnemy.HP}') #DEBUG  
                    pass #Do nothing
                elif activePlayer.HP > 0 and activeEnemy.HP <= 0:
                    skirmishResults = 'Win'
                    self.recordData(activePlayer, activeEnemy, playedCards, skirmishResults)
                    if activePlayer.name == self.player.name:
                        self.enemyList.pop(0) #Remove enemy from enemy list    
                        self.killTarget -= 1 
                        #print(f'Kill Target: {self.killTarget}\n') #DEBUG
                        #print(f'{activePlayer.name} Wins this round!') #DEBUG
                        #print(f'Active Player: {activePlayer.name}\tHP: {activePlayer.HP}') #DEBUG
                        #print(f'Active Enemy: {activeEnemy.name}\tHP: {activeEnemy.HP}') #DEBUG
                    else:
                        #print(f'{activePlayer.name} Wins this round!') #DEBUG
                        #print(f'Active Player: {activePlayer.name}\tHP: {activePlayer.HP}') #DEBUG
                        #print(f'Active Enemy: {activeEnemy.name}\tHP: {activeEnemy.HP}') #DEBUG
                        break #Because human player is dead at this point
                elif activePlayer.HP <= 0 and activeEnemy.HP > 0:
                    skirmishResults = 'Loss'
                    self.recordData(activePlayer, activeEnemy, playedCards, skirmishResults)
                    #print(f'{activePlayer.name} Loses this round!') #DEBUG
                    #print(f'Active Player: {activePlayer.name}\tHP: {activePlayer.HP}') #DEBUG
                    #print(f'Active Enemy: {activeEnemy.name}\tHP: {activeEnemy.HP}') #DEBUG
                    #print(f'Kill Target: {self.killTarget}\n') #DEBUG                               
                    break
                elif activePlayer.HP <= 0 and activeEnemy.HP <= 0:
                    skirmishResults = 'Loss' 
                    self.recordData(activePlayer, activeEnemy, playedCards, skirmishResults)
                    #print(f'{activePlayer.name} Loses this round!') #DEBUG
                    #print(f'Active Player: {activePlayer.name}\tHP: {activePlayer.HP}') #DEBUG
                    #print(f'Active Enemy: {activeEnemy.name}\tHP: {activeEnemy.HP}') #DEBUG
                    #print(f'Kill Target: {self.killTarget}\n') #DEBUG                               
                    break
                else:
                    raise Exception('Something went wrong with the Player or Enemy HP!')

                #Attack phase
                if activePlayer.WPN == None:
                    activeEnemy.HP -= activePlayer.STR
                    #print(f'{activePlayer.name} did {activePlayer.STR} damage to {activeEnemy.name}')

                else:
                    activeEnemy.HP -= (activePlayer.STR*activePlayer.WPN.WPN_factor)
                    #print(f'{activePlayer.name} did {(activePlayer.STR*activePlayer.WPN.WPN_factor)} damage to {activeEnemy.name}')

                #Health check
                if activePlayer.HP > 0 and activeEnemy.HP > 0:
                    skirmishResults = 'Draw'
                    self.recordData(activePlayer, activeEnemy, playedCards, skirmishResults)
                    #print(f'Active Player: {activePlayer.name}\tHP: {activePlayer.HP}') #DEBUG
                    #print(f'Active Enemy: {activeEnemy.name}\tHP: {activeEnemy.HP}') #DEBUG                        
                    pass #Do nothing
                elif activePlayer.HP > 0 and activeEnemy.HP <= 0:
                    skirmishResults = 'Win'
                    self.recordData(activePlayer, activeEnemy, playedCards, skirmishResults)
                    if activePlayer.name == self.player.name:
                        if len(self.enemyList) == 0:
                            break #might have to decrease killTarget here
                        else:
                            self.enemyList.pop(0) #Remove enemy from enemy list    
                            self.killTarget -= 1 
                            #print(f'Kill Target: {self.killTarget}\n') #DEBUG
                            #print(f'{activePlayer.name} Wins this round!') #DEBUG
                            #print(f'Active Player: {activePlayer.name}\tHP: {activePlayer.HP}') #DEBUG
                            #print(f'Active Enemy: {activeEnemy.name}\tHP: {activeEnemy.HP}') #DEBUG
                    else:
                        #print(f'{activePlayer.name} Wins this round!') #DEBUG
                        #print(f'Active Player: {activePlayer.name}\tHP: {activePlayer.HP}') #DEBUG
                        #print(f'Active Enemy: {activeEnemy.name}\tHP: {activeEnemy.HP}') #DEBUG
                        break #Because human player is dead at this point
                elif activePlayer.HP <= 0 and activeEnemy.HP > 0:
                    skirmishResults = 'Loss'
                    self.recordData(activePlayer, activeEnemy, playedCards, skirmishResults)
                    #print(f'{activePlayer.name} Loses this round!') #DEBUG
                    #print(f'Active Player: {activePlayer.name}\tHP: {activePlayer.HP}') #DEBUG
                    #print(f'Active Enemy: {activeEnemy.name}\tHP: {activeEnemy.HP}') #DEBUG
                    #print(f'Kill Target: {self.killTarget}\n') #DEBUG                               
                    break
                elif activePlayer.HP <= 0 and activeEnemy.HP <= 0:
                    skirmishResults = 'Loss'
                    self.recordData(activePlayer, activeEnemy, playedCards, skirmishResults)
                    #print(f'{activePlayer.name} Loses this round!') #DEBUG
                    #print(f'Active Player: {activePlayer.name}\tHP: {activePlayer.HP}') #DEBUG
                    #print(f'Active Enemy: {activeEnemy.name}\tHP: {activeEnemy.HP}') #DEBUG
                    #print(f'Kill Target: {self.killTarget}\n') #DEBUG                               
                    break
                else:
                    raise Exception('Something went wrong with the Player or Enemy HP!')
            
            else:
                raise Exception('WARNING: Player or Enemy HP less than zero!')                
         
            #Alternate turns
            if self.playerTurn == True:
                self.playerTurn = False
            elif self.playerTurn == False:
                self.playerTurn = True  
            
            #print('\n')

def runSimulation(n_runs=10):
    games = [GameLogic() for i in range(n_runs)] #Run simulation
    enc = [game.encounter for game in games] #Capture encounters

    df_encounters = pd.DataFrame(None)

    for i in enc:
        df_encounters = pd.concat([df_encounters, i], axis=0, ignore_index=True)

    return df_encounters

def AnalyseConstants(k):
    player_STR = [p.STR for p in k.players]
    minion_STR = [m.STR for m in k.minions]
    boss_STR = [b.STR for b in k.bosses]
    
    player_HP = [p.HP for p in k.players]
    minion_HP = [m.HP for m in k.minions]
    boss_HP = [b.HP for b in k.minions]
    
    print(f'Player/Minion/Boss STR: Mean {round(np.mean(player_STR),2)}/{round(np.mean(minion_STR),2)}/{round(np.mean(boss_STR),2)} | StDev {round(np.std(player_STR),2)}/{round(np.std(minion_STR),2)}/{round(np.std(boss_STR),2)}')
    print(f'Player vs. Minion/Boss STR: {round(np.mean(player_STR)-np.mean(minion_STR),2)}/{round(np.mean(player_STR)-np.mean(boss_STR),2)}')

    print(f'\nPlayer/Minion/Boss HP: Mean {round(np.mean(player_HP),2)}/{round(np.mean(minion_HP),2)}/{round(np.mean(boss_HP),2)} | StDev {round(np.std(player_HP),2)}/{round(np.std(minion_HP),2)}/{round(np.std(boss_HP),2)}')
    print(f'Player vs. Minion/Boss HP: {round(np.mean(player_HP)-np.mean(minion_HP),2)}/{round(np.mean(player_HP)-np.mean(boss_HP),2)}')
    
    print('\nPlayers:', [(p.name, p.HP) for p in k.players])
    print('Minions:', [(p.name, p.HP) for p in k.minions])
    print('Bosses:', [(p.name, p.HP) for p in k.bosses])

print('Classes initialized!') #DEBUG

Classes initialized!


In [2]:
#Batch number
batch = 10

#Run simulation to generate data
game_data = runSimulation(n_runs=30000)

#Output game data to csv for future analysis
game_data.to_csv(f'Game_Data_{batch}.csv', sep='\t', index=False, encoding='utf-8')

print('Rows of Data:', len(game_data))

Rows of Data: 151290


In [ ]:
#Debug constants --> show all constants and their relationship
DEBUG_constants = GameLogic()
AnalyseConstants(DEBUG_constants.constants)

First run: No change to constants. 100k+ rows. No weapons.
Second run: No change to constants. 100k+ rows. No weapons.
Third run: No change to k. 120k+ rows. Weapons.
Fourth run: Various modifications to k; HP and STR only. 129k+ rows. Weapons.
Fifth run: No change to k. Chg Minion range from 4-7 to 3-6. 
Sixth run: No change to k. Chg Minion range from 3-6 to 3-5.
Seventh run: Chg "Nice night for a walk" value from -1000 to -15.
Eighth run: Chg Minion range from 3-5 to 2-4. No change to k.
Ninth run: Added +5 to all hero HP.
Tenth run: Added +5 to all hero STR.